In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Carregar os dados de geração e clima
generation_data = pd.read_csv('Plant_2_Generation_Dataset.csv')
weather_data = pd.read_csv('Plant_2_Weather_Sensor_Data.csv')

# Verificar se ambos os DataFrames foram carregados corretamente
print("Geração de dados:", generation_data.head())
print("\nDados climáticos:", weather_data.head())

# Converter 'DATE_TIME' para datetime
generation_data['DATE_TIME'] = pd.to_datetime(generation_data['DATE_TIME'])
weather_data['DATE_TIME'] = pd.to_datetime(weather_data['DATE_TIME'])

# Mesclar os dados com base na coluna 'DATE_TIME'
df_solar = pd.merge(
    generation_data.drop(columns=['PLANT_ID']),
    weather_data.drop(columns=['PLANT_ID', 'SOURCE_KEY']),
    on='DATE_TIME'
)

# Extrair a data e a hora
df_solar['DATE'] = df_solar['DATE_TIME'].dt.date
df_solar['HOUR'] = df_solar['DATE_TIME'].dt.hour

# Obter a produção máxima diária (DC_POWER máximo por dia)
daily_max_production = df_solar.groupby('DATE')['DC_POWER'].max().reset_index()
daily_max_production.rename(columns={'DC_POWER': 'MAX_DAILY_DC_POWER'}, inplace=True)

# Obter a temperatura média do dia
daily_avg_temp = df_solar.groupby('DATE')['AMBIENT_TEMPERATURE'].mean().reset_index()
daily_avg_temp.rename(columns={'AMBIENT_TEMPERATURE': 'AVG_DAILY_TEMPERATURE'}, inplace=True)

# Obter a temperatura média do módulo no dia
daily_avg_module_temp = df_solar.groupby('DATE')['MODULE_TEMPERATURE'].mean().reset_index()
daily_avg_module_temp.rename(columns={'MODULE_TEMPERATURE': 'AVG_DAILY_MODULE_TEMPERATURE'}, inplace=True)

# Determinar o tempo ensolarado por dia (quantidade de horas com irradiação > 0)
df_solar['SUNNY'] = df_solar['IRRADIATION'] > 0
sunny_hours = df_solar[df_solar['SUNNY']].groupby('DATE')['HOUR'].nunique().reset_index()
sunny_hours.rename(columns={'HOUR': 'SUNNY_DURATION'}, inplace=True)

# Calcular a porcentagem de sol
total_hours_day = 24  # Total de horas no dia
sunny_hours['SUNNY_PERCENTAGE'] = (sunny_hours['SUNNY_DURATION'] / total_hours_day) * 100

# Obter o rendimento total diário (TOTAL_YIELD no último registro do dia)
daily_total_yield = df_solar.groupby('DATE')['TOTAL_YIELD'].max().reset_index()
daily_total_yield.rename(columns={'TOTAL_YIELD': 'TOTAL_DAILY_YIELD'}, inplace=True)

# Combinar todos os dados em um único DataFrame
daily_data = daily_max_production.merge(daily_avg_temp, on='DATE')
daily_data = daily_data.merge(daily_avg_module_temp, on='DATE')
daily_data = daily_data.merge(sunny_hours[['DATE', 'SUNNY_PERCENTAGE']], on='DATE')
daily_data = daily_data.merge(daily_total_yield, on='DATE')

# Verificar o DataFrame combinado
print("\nDados Diários Combinados:", daily_data.head())

# Preparar os dados para treinamento
X = daily_data[['MAX_DAILY_DC_POWER', 'AVG_DAILY_TEMPERATURE', 'AVG_DAILY_MODULE_TEMPERATURE', 'SUNNY_PERCENTAGE']]
y = daily_data['TOTAL_DAILY_YIELD']  # Supondo que esteja em kWh

# Tratar valores ausentes (se houver)
X = X.fillna(0)

# Dividir os dados em conjuntos de treinamento e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar o modelo de regressão linear
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

# Avaliar o modelo usando R²
from sklearn.metrics import r2_score
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f"R² do modelo: {r2:.2f}")

# Avaliar o modelo usando F1 Score (Nota: Não é comum para regressão)
# Removemos essa métrica, pois não é apropriada para regressão.
# Em vez disso, podemos usar métricas como MAE ou RMSE.

from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"MAE do modelo: {mae:.2f}")
print(f"RMSE do modelo: {rmse:.2f}")

# Salvar o modelo em um arquivo pickle
import pickle
with open('modelo_solar.pkl', 'wb') as file:
    pickle.dump(model, file)

Geração de dados:              DATE_TIME  PLANT_ID       SOURCE_KEY  DC_POWER  AC_POWER  \
0  2020-05-15 00:00:00   4136001  4UPUqMRk7TRMgml       0.0       0.0   
1  2020-05-15 00:00:00   4136001  81aHJ1q11NBPMrL       0.0       0.0   
2  2020-05-15 00:00:00   4136001  9kRcWv60rDACzjR       0.0       0.0   
3  2020-05-15 00:00:00   4136001  Et9kgGMDl729KT4       0.0       0.0   
4  2020-05-15 00:00:00   4136001  IQ2d7wF4YD8zU1Q       0.0       0.0   

   DAILY_YIELD   TOTAL_YIELD  
0  9425.000000  2.429011e+06  
1     0.000000  1.215279e+09  
2  3075.333333  2.247720e+09  
3   269.933333  1.704250e+06  
4  3177.000000  1.994153e+07  

Dados climáticos:              DATE_TIME  PLANT_ID       SOURCE_KEY  AMBIENT_TEMPERATURE  \
0  2020-05-15 00:00:00   4136001  iq8k7ZNt4Mwm3w0            27.004764   
1  2020-05-15 00:15:00   4136001  iq8k7ZNt4Mwm3w0            26.880811   
2  2020-05-15 00:30:00   4136001  iq8k7ZNt4Mwm3w0            26.682055   
3  2020-05-15 00:45:00   4136001  iq8k7ZNt

In [ ]:
from flask import Flask, jsonify, request

app = Flask(__name__)

@app.route('/predict/', methods=['POST'])
def prever():
    data = request.get_json()
    
    # Obter os dados de entrada
    plate_power = data.get('plate_power', '')
    daily_irradiation = data.get('daily_irradiation', '')
    system_efficiency = data.get('system_efficiency', '')
    
    # Validar e converter os dados de entrada para float
    try:
        plate_power = float(plate_power)
        daily_irradiation = float(daily_irradiation)
        system_efficiency = float(system_efficiency)
    except ValueError:
        return jsonify({'error': 'Dados de entrada inválidos. Certifique-se de que todos os campos são numéricos.'}), 400
    
    # Calcular a energia total gerada
    total_generated = plate_power * daily_irradiation * system_efficiency
    
    # Retornar o resultado
    return jsonify({'total_generated': total_generated})

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1